In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
from scipy.optimize import fsolve
from scipy.integrate import odeint, quad
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from scipy.optimize import minimize_scalar as sc
from iapws import IAPWS97 as steam
import os
from spectrum_class import Spectrum
from molarity_class import Molarity
from diagnostic_class import Diagnostic

In [2]:
# ----- Obtain the path to the spa folder
path = os.path.dirname(os.getcwd())

path_spa = path + '\\spa data\\'

In [4]:
spa = os.listdir(path_spa)

# Loop through each folder in the spa folder
for folder in spa:
    # Construct the full path to the inner folders
    folder_path = os.path.join(path_spa, folder)
    
    # make a list of all of the folders inside the spa folder
    files = os.listdir(folder_path)

    # loop through
    for file in folder_path:
        file_path = os.path.join(folder_path,file)
        print(file_path)

c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\c
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\:
c:\
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\U
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\s
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\e
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\r
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\s
c:\
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\M
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\o
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\d
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\e
c:\Users\Modes Lab\Desktop\MODES\Battery Data\Li-Li Symmetric Cell\spa data\0M\s
c:\Users\Modes Lab\D

In [5]:
path_spa = path + '\\spa data\\'

# Initialize an empty dictionary to store data
data_dict = {}

# Loop through each inner folder
for molarity_folder in os.listdir(path_spa):
    # Construct the full path to the inner folder
    molarity_folder = os.path.join(path_spa, molarity_folder)
    
    # Check if the path is a directory
    if os.path.isdir(molarity_folder):
        # Initialize an empty list to store data from files in this inner folder
        spectra = []
        
        # Loop through each file in the inner folder
        for file_name in os.listdir(molarity_folder):
            # Construct the full path to the file
            file_path = os.path.join(molarity_folder, file_name)
            
            # Read data from the file using pandas
            # Adjust parameters according to your file format (e.g., delimiter, header, etc.)
            data = pd.read_csv(file_path,sep = '\t')
            
            # Append the data to the list for this inner folder
            spectra.append(data)
        
        # Store the list of data from this inner folder in the dictionary
        data_dict[molarity_folder] = spectra

# Now data_dict contains data from each inner folder, with inner folder names as keys
display(data_dict)


{'c:\\Users\\Modes Lab\\Desktop\\MODES\\Battery Data\\Li-Li Symmetric Cell\\tsv data\\.4M Data': [                X         Y
  0      650.000000  0.311604
  1      650.482084  0.311588
  2      650.964168  0.314802
  3      651.446251  0.316489
  4      651.928335  0.318061
  ...           ...       ...
  6945  3998.071665  0.219124
  6946  3998.553749  0.219395
  6947  3999.035832  0.219718
  6948  3999.517916  0.220085
  6949  4000.000000  0.220138
  
  [6950 rows x 2 columns],
                  X         Y
  0      650.000000  0.350967
  1      650.482084  0.350964
  2      650.964168  0.353470
  3      651.446251  0.354745
  4      651.928335  0.355919
  ...           ...       ...
  6945  3998.071665  0.235167
  6946  3998.553749  0.235377
  6947  3999.035832  0.235614
  6948  3999.517916  0.235879
  6949  4000.000000  0.235917
  
  [6950 rows x 2 columns],
                  X         Y
  0      650.000000  0.387539
  1      650.482084  0.387590
  2      650.964168  0.390624
  3 